In [1]:
import tensorflow as tf
import numpy as np
import pymysql
import datetime
from datetime import date, timedelta
import configparser

cf = configparser.ConfigParser()
cf.read('config.cfg')
                               
DB_IP = cf.get('db', 'DB_IP')
DB_USER = cf.get('db', 'DB_USER')
DB_PWD = cf.get('db', 'DB_PWD')
DB_SCH = cf.get('db', 'DB_SCH')

LIMIT_FILTER = 0.70

INPUT_VEC_SIZE = 7
LSTM_SIZE = 7
TIME_STEP_SIZE = 60
LABEL_SIZE = 3
LSTM_DEPTH = 4

BATCH_SIZE = 15000
TRAIN_CNT = 600

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [2]:
class DBManager :
    def __init__(self):
        self.conn = self.get_new_conn()
        
    def __del__(self):
        self.conn.close()
    def get_new_conn(self):
        return pymysql.connect(host=DB_IP, user=DB_USER, password=DB_PWD, db=DB_SCH, charset='utf8mb4')
    def get_codedates(self, code, limit):    
        query = "SELECT date FROM data.daily_stock WHERE code = %s AND date <= %s ORDER BY date ASC"
        cursor = self.conn.cursor()
        cursor.execute(query, (code, limit))
        code_dates = list()        
        dates = cursor.fetchall()
        for date in dates:
            code_dates.append((code, date[0]))
        return code_dates
    def get_items(self, code, date, limit):
        query = "SELECT open, high, low, close, volume, hold_foreign, st_purchase_inst FROM data.daily_stock WHERE code = %s AND date >= %s ORDER BY date ASC LIMIT %s"
        cursor = self.conn.cursor()
        cursor.execute(query, (code, date, limit))
        items = cursor.fetchall()        
        return items
    
    def get_codes(self):
        query = "SELECT DISTINCT code FROM data.daily_stock"
        cursor = self.conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    def insert_result(self, expect, code, analyze_at, potential, evaluate, volume) :
        if self.check_exist(expect, code, analyze_at, evaluate):
            print('duplicate', expect, code, analyze_at)
        else :
            cursor = self.conn.cursor()
            print(expect,code,analyze_at,potential,volume,evaluate)
            cursor.execute("INSERT INTO forecast (type, code, analyzeAt, potential, volume, evaluate, train) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                           (expect, code, analyze_at, str(potential), volume, evaluate, TRAIN_CNT))
            self.conn.commit()
    def check_exist(self, expect, code, analyze_at, evaluate):
        cursor = self.conn.cursor()
        cursor.execute("SELECT count(*) as cnt FROM forecast WHERE type = %s AND code = %s AND analyzeAt = %s AND evaluate = %s AND train=%s", (expect, code, analyze_at, evaluate, TRAIN_CNT))
        cnt = cursor.fetchone()
        return cnt[0] > 0
    def get_volume(self, code, limit_at):
        cursor = self.conn.cursor()
        cursor.execute("SELECT count(*) as cnt FROM daily_stock WHERE code = %s AND date <= %s", (code, limit_at))
        cnt = cursor.fetchone()
        return cnt[0]
        
    def get_last_date_at(self):
        cursor = self.conn.cursor()
        cursor.execute("select distinct(date) from data.daily_stock order by date desc limit 1")
        last = cursor.fetchone()
        return last[0]
    def get_last_analyze_at(self):
        cursor = self.conn.cursor()
        cursor.execute("select distinct(analyzeAt) from data.forecast order by analyzeAt desc limit 1")
        last = cursor.fetchone()
        return last[0]

In [3]:
def read_series_datas(db, code_dates):
    X = list()
    Y = list()
    for code_date in code_dates:
        items = db.get_items(code_date[0], code_date[1], TIME_STEP_SIZE + EVALUATE_SIZE)
  
        if len(items) < (EVALUATE_SIZE + TIME_STEP_SIZE):
            break
        X.append(np.array(items[:TIME_STEP_SIZE]))

        st_purchase_inst = items[-(EVALUATE_SIZE + 1)][EXPECT]
        if st_purchase_inst == 0:
            continue
        for i in range(EVALUATE_SIZE, len(items) - EVALUATE_SIZE):
            eval_inst = items[i][EXPECT]
            eval_bef = items[EVALUATE_SIZE-i][EXPECT]
            if eval_bef < eval_inst:
                eval_bef = eval_inst           
        
        if (eval_bef - st_purchase_inst) / st_purchase_inst < -0.02: #percent ? cnt ? 
            Y.append((0., 0., 1.))
        elif (eval_bef - st_purchase_inst) / st_purchase_inst > 0.03:
            Y.append((1., 0., 0.))
        else:
            Y.append((0., 1., 0.))


    arrX = np.array(X)    
    meanX = np.mean(arrX, axis = 0)
    stdX = np.std(arrX, axis = 0)
    norX = (arrX - meanX) / stdX
    norY = np.array(Y)
    return norX, norY


In [4]:
def read_datas(db, code_dates):    
    np.random.seed()
    np.random.shuffle(code_dates)

    trX = list()
    trY = list()
    trX, trY = read_series_datas(db, code_dates)
    teX, teY = read_series_datas(db, code_dates)

    return trX, trY, teX, teY

In [5]:
def analyze(code, limit):      
    db = DBManager()
    code_dates = db.get_codedates(code, limit)
    tf.reset_default_graph()    
    last = code_dates[-1][1]
    trX, trY, teX, teY = read_datas(db, code_dates)
    if (len(trX) == 0):
        return None

    X = tf.placeholder(tf.float32, [None, TIME_STEP_SIZE, INPUT_VEC_SIZE])
    Y = tf.placeholder(tf.float32, [None, LABEL_SIZE])

    W = init_weights([LSTM_SIZE, LABEL_SIZE])
    B = init_weights([LABEL_SIZE])
    py_x, state_size = model(code, X, W, B, LSTM_SIZE)

    loss = tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y)
    cost = tf.reduce_mean(loss)
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    predict_op = tf.argmax(py_x, 1)

    # Launch the graph in a session
    analyzed = None
    with tf.Session() as sess:
        
        tf.global_variables_initializer().run()

        for loop in range(TRAIN_CNT):
            for start, end in zip(range(0, len(trX), BATCH_SIZE), range(BATCH_SIZE, len(trX)+1, BATCH_SIZE)):
                sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

            test_indices = np.arange(len(teY))
            org = teY[test_indices] ## fixfix
            res = sess.run(predict_op, feed_dict={X: teX[test_indices], Y: teY[test_indices]})
            
            if loop == TRAIN_CNT-1 :
                result = np.mean(np.argmax(org, axis=1) == res)                
                analyzed = {"code":code, "per":round(result, 2), "date":limit}
    return analyzed


In [6]:
def model(code, X, W, B, lstm_size):
    XT = tf.transpose(X, [1, 0, 2]) 
    XR = tf.reshape(XT, [-1, lstm_size])
    #ValueError: Tensor conversion requested dtype int32 for Tensor with dtype
    #float32: 'Tensor("Reshape:0", shape=(?, 7), dtype=float32)'
#tf.cast(input_y, tf.float32) or tf.to_float(input_y).
    X_split = tf.split(XR, num_or_size_splits=TIME_STEP_SIZE, axis=0)
    with tf.variable_scope(code, reuse=False):
        #cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        cell = tf.contrib.rnn.GRUCell(lstm_size)
        cell = tf.contrib.rnn.DropoutWrapper(cell = cell, output_keep_prob = 0.5)
        cell = tf.contrib.rnn.MultiRNNCell([cell] * LSTM_DEPTH, state_is_tuple = True)
    outputs, _states = tf.contrib.rnn.static_rnn(cell, X_split, dtype=tf.float32)

    return tf.matmul(outputs[-1], W) + B, cell.state_size # State size to initialize the stat

In [7]:
def get_analyze_at_date(target_date):
    analyze_at = target_date + timedelta(days=EVALUATE_SIZE)
    return analyze_at

In [ ]:
target_at = datetime.datetime.strptime('20170101', '%Y%m%d') #DBManager().get_last_analyze_at() - timedelta(days=EVALUATE_SIZE)
loop_size = timedelta(days=1)
limit_at = DBManager().get_last_date_at().date()

EXPECT = 3 ##open, high, low, close, volume, hold_foreign, st_purchase_inst
EVALUATE_SIZE = 3
#target_at = datetime.datetime.strptime(limit, '%Y-%m-%d') + timedelta(days=EVALUATE_SIZE)

while limit_at > target_at.date():    
    target_at += loop_size
    db = DBManager()
    codes = db.get_codes()
    for code in codes :
        analyzed = analyze(code[0], target_at)
        analyze_at = get_analyze_at_date(target_at)
        if analyzed is None or db.check_exist(EXPECT,  analyzed["code"], analyze_at, EVALUATE_SIZE):
            continue
        code = analyzed["code"]
        db = DBManager()
        volume = db.get_volume(code, target_at)    
        db.insert_result(EXPECT, code, analyze_at, analyzed["per"], EVALUATE_SIZE, volume)        

    print('done')

3 A000030 2017-01-05 00:00:00 0.25 524 3
3 A000050 2017-01-05 00:00:00 0.8 3226 3
3 A000070 2017-01-05 00:00:00 0.36 3226 3
3 A000080 2017-01-05 00:00:00 0.09 1786 3
3 A000100 2017-01-05 00:00:00 0.22 3226 3
3 A000120 2017-01-05 00:00:00 0.5 3226 3
3 A000140 2017-01-05 00:00:00 0.09 3226 3
3 A000150 2017-01-05 00:00:00 0.0 3226 3
3 A000210 2017-01-05 00:00:00 0.13 3226 3
3 A000230 2017-01-05 00:00:00 0.51 3226 3
3 A000240 2017-01-05 00:00:00 0.43 3226 3
3 A000270 2017-01-05 00:00:00 0.5 3226 3
3 A000640 2017-01-05 00:00:00 0.0 3226 3
3 A000660 2017-01-05 00:00:00 0.67 3226 3
3 A000670 2017-01-05 00:00:00 0.56 3226 3
3 A000720 2017-01-05 00:00:00 0.04 3226 3
3 A000810 2017-01-05 00:00:00 0.34 3226 3
3 A000880 2017-01-05 00:00:00 0.51 3226 3
3 A000990 2017-01-05 00:00:00 0.47 3226 3
3 A001040 2017-01-05 00:00:00 0.11 3226 3
3 A001060 2017-01-05 00:00:00 0.52 3226 3
3 A001120 2017-01-05 00:00:00 0.0 3226 3
3 A001230 2017-01-05 00:00:00 0.48 3226 3
3 A001430 2017-01-05 00:00:00 0.21 3226 3

c:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:31: RuntimeWarning: invalid value encountered in true_divide


3 A006260 2017-01-05 00:00:00 0.0 3226 3
3 A006280 2017-01-05 00:00:00 0.14 3226 3
3 A006400 2017-01-05 00:00:00 0.26 3226 3
3 A006650 2017-01-05 00:00:00 0.31 3226 3
3 A006800 2017-01-05 00:00:00 0.5 3226 3
3 A007070 2017-01-05 00:00:00 0.22 1240 3
3 A007210 2017-01-05 00:00:00 0.44 3226 3
3 A007310 2017-01-05 00:00:00 0.47 3226 3
3 A007340 2017-01-05 00:00:00 0.09 3226 3
3 A007570 2017-01-05 00:00:00 0.15 3226 3
3 A007690 2017-01-05 00:00:00 0.45 3226 3
3 A008060 2017-01-05 00:00:00 0.11 3226 3
3 A008490 2017-01-05 00:00:00 0.19 3226 3
3 A008560 2017-01-05 00:00:00 0.54 3226 3
3 A008770 2017-01-05 00:00:00 0.27 3226 3
3 A008930 2017-01-05 00:00:00 0.51 3226 3
3 A009150 2017-01-05 00:00:00 0.34 3226 3
3 A009240 2017-01-05 00:00:00 0.02 3226 3
3 A009290 2017-01-05 00:00:00 0.59 3226 3
3 A009420 2017-01-05 00:00:00 0.07 3226 3
3 A009540 2017-01-05 00:00:00 0.08 3226 3
3 A009830 2017-01-05 00:00:00 0.46 3226 3
3 A010060 2017-01-05 00:00:00 0.35 3226 3
3 A010120 2017-01-05 00:00:00 0.4 32

c:\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


3 A023530 2017-01-05 00:00:00 0.38 2701 3
3 A024110 2017-01-05 00:00:00 0.26 3226 3
3 A025540 2017-01-05 00:00:00 0.14 3226 3
3 A025860 2017-01-05 00:00:00 0.51 3226 3
3 A027410 2017-01-05 00:00:00 0.0 648 3
3 A028050 2017-01-05 00:00:00 0.44 3226 3
3 A028260 2017-01-05 00:00:00 0.0 503 3
3 A029530 2017-01-05 00:00:00 0.66 3226 3
3 A029780 2017-01-05 00:00:00 0.57 2360 3
3 A030000 2017-01-05 00:00:00 0.27 3226 3
3 A030200 2017-01-05 00:00:00 0.32 3226 3
3 A032640 2017-01-05 00:00:00 0.37 3226 3
3 A032830 2017-01-05 00:00:00 0.71 1645 3
3 A033780 2017-01-05 00:00:00 0.43 3226 3
3 A033920 2017-01-05 00:00:00 0.66 3226 3
3 A034020 2017-01-05 00:00:00 0.17 3226 3
3 A034120 2017-01-05 00:00:00 0.52 3226 3
3 A034220 2017-01-05 00:00:00 0.43 3089 3
3 A034730 2017-01-05 00:00:00 0.2 1769 3
3 A035250 2017-01-05 00:00:00 0.4 3226 3
3 A035420 2017-01-05 00:00:00 0.46 3226 3
3 A036460 2017-01-05 00:00:00 0.06 3226 3
3 A036570 2017-01-05 00:00:00 0.38 3226 3
3 A036580 2017-01-05 00:00:00 0.6 3226 3

In [ ]:
db = DBManager()
cursor = db.conn.cursor()
cursor.execute("SELECT id, evaluate, analyzeAt FROM data.forecast order by analyzeAt desc;")
results = cursor.fetchall()
for result in range(1):
    roll = result[2] - timedelta(days=result[1])
    print(roll)                                 
    cursor.execute("update data.forecast set analyzeAt=%s where id=%s", (roll, result[0]))